In [1]:
from extractors.pdf_extractor import extract_pdf_as_markdown,extract_docx_as_markdown
from chains.audit_chain import analyze_module
from chains.evidence_chain import analyze_evidence
from chains.improvement_chain import get_improvement_chain
from langchain_community.chat_models import ChatOpenAI
import pandas as pd
from dotenv import load_dotenv
import asyncio
import re
import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

In [2]:
load_dotenv()
# print(os.getenv("OPENAI_API_KEY"))

file_path = "./files/"
controls = pd.read_json("ISO_27001_2022_Controls_List.json")


In [3]:
controls

,Section,Control ID,Control Title,Clause
0,A.5 (Organisational Controls),A.5.1,Policies for information security,"4,5"
1,A.5 (Organisational Controls),A.5.2,Information security roles and responsibilities,"4,5"
2,A.5 (Organisational Controls),A.5.3,Segregation of duties,5
3,A.5 (Organisational Controls),A.5.4,Management responsibilities,"5,6,9,10"
4,A.5 (Organisational Controls),A.5.5,Contact with authorities,
...,...,...,...,...
88,A.8 (Technological Controls),A.8.30,Outsourced development,
89,A.8 (Technological Controls),A.8.31,"Separation of development, test and production...",6
90,A.8 (Technological Controls),A.8.32,Change management,8
91,A.8 (Technological Controls),A.8.33,Test information,


In [4]:
import numpy as np

n = len(controls)
part_size = n // 5
remainder = n % 5

# Calculate split indices
sizes = [part_size + (1 if i < remainder else 0) for i in range(5)]
indices = np.cumsum([0] + sizes)

dfs = [controls.iloc[indices[i]:indices[i+1]].reset_index(drop=True) for i in range(5)]
df1, df2, df3, df4, df5 = dfs

In [5]:
def extract_file_as_markdown(file_path):
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".pdf":
        return extract_pdf_as_markdown(file_path)
    elif ext == ".docx":
        return extract_docx_as_markdown(file_path)
    else:
        raise ValueError(f"Unsupported file type: {ext}")

In [6]:
def read_folder_and_join_markdown(folder_path, file_list=None):
    """
    Args:
        folder_path (str): Root folder to search.
        file_list (list[str], optional): List of filenames (with or without relative paths) to include.
                                         If None, include all .pdf/.docx files.

    Returns:
        str: Combined markdown string from all valid files.
    """
    all_markdown = []
    normalized_file_list = set(os.path.normpath(f).lower() for f in file_list) if file_list else None
    print(f"Normalized file list: {normalized_file_list}")

    for root, _, files in os.walk(folder_path):
        for filename in files:
            ext = os.path.splitext(filename)[1].lower()
            if ext not in [".pdf", ".docx"]:
                continue

            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, folder_path)
            normalized_relative_path = os.path.normpath(relative_path).lower()

            if normalized_file_list and normalized_relative_path not in normalized_file_list:
                continue  # Skip files not in the list

            try:
                markdown = extract_file_as_markdown(file_path)
                all_markdown.append(f"## File: {relative_path}\n\n{markdown}")
            except Exception as e:
                print(f"❌ Error processing {file_path}: {e}")

    return "\n\n---\n\n".join(all_markdown)
def read_folder_and_join_markdown_exclude(folder_path, exclude_file_list=None):
    """
    Args:
        folder_path (str): Root folder to search.
        exclude_file_list (list[str], optional): List of file paths (relative to folder_path) to exclude.

    Returns:
        str: Combined markdown string from all valid, non-excluded files.
    """
    all_markdown = []
    normalized_exclude_list = set(os.path.normpath(f).lower() for f in exclude_file_list) if exclude_file_list else set()

    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.startswith("._"):
                continue  
            ext = os.path.splitext(filename)[1].lower()
            if ext not in [".pdf", ".docx"]:
                continue

            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, folder_path)
            normalized_relative_path = os.path.normpath(relative_path).lower()

            if normalized_relative_path in normalized_exclude_list:
                continue  # Skip files in the exclusion list

            try:
                markdown = extract_file_as_markdown(file_path)
                all_markdown.append(f"## File: {relative_path}\n\n{markdown}")
            except Exception as e:
                print(f"❌ Error processing {file_path}: {e}")

    return "\n\n---\n\n".join(all_markdown)

In [7]:
file_list = ["Business Continuity Plan E-WISE [Restricted].docx.pdf","Information Security Policy [Internal].docx","Initial Implementation Plan - AI-Driven Document Compliance Analysis System (3).pdf"]
# file_list = ["Business Continuity Plan E-WISE [Restricted].docx.pdf","Information Security Policy [Internal].docx"]
evidence_text = read_folder_and_join_markdown_exclude(file_path, exclude_file_list=file_list)
text = read_folder_and_join_markdown(file_path,file_list=file_list)


✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1
Normalized file list: {'information security policy [internal].docx', 'business continuity plan e-wise [restricted].docx.pdf', 'initial implementation plan - ai-driven document compliance analysis system (3).pdf'}
✔️ Tesseract detected: tesseract 4.1.1
✔️ Tesseract detected: tesseract 4.1.1


In [8]:
len(text)

128612

In [9]:

# llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3, api_key=os.getenv("OPENAI_API_KEY"))
# clauses = [df1,df2,df3,df4,df5]
# audits = []
# results = []
# for clause in clauses:
#     audit_chain = analyze_module(llm)
#     results.append(audit_chain.run(text=text, control_json=clause.to_json(orient='records', indent=2)))



In [10]:
from langchain.callbacks import get_openai_callback
import nest_asyncio
import sys
nest_asyncio.apply()

# text = extract_pdf_as_markdown(file_path)

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))
clauses = [df1,df2,df3,df4,df5]
results = []
total_tokens = 0
total_cost = 0.0

async def process_clause_async(clause, text, llm):
    def sync_callback_wrapper():
        with get_openai_callback() as cb:
            audit_chain = analyze_module(llm)
            result = audit_chain.run(
                text=text,
                control_json=clause.to_json(orient='records', indent=2)
            )
            return {
                "result": result,
                "tokens": cb.total_tokens,
                "cost": cb.total_cost,
                "controls": len(clause)
            }

    return await asyncio.to_thread(sync_callback_wrapper)

async def run_all_clauses():
    results = []
    total_tokens = 0
    total_cost = 0.0

    tasks = [process_clause_async(clause, text, llm) for clause in clauses]
    clause_results = await asyncio.gather(*tasks)

    for data in clause_results:
        results.append(data["result"])
        total_tokens += data["tokens"]
        total_cost += data["cost"]
        print(f"🔎 Clause Processed: {data['controls']} controls")
        print(f"🧠 Tokens used: {data['tokens']}")
        print(f"💵 Cost: ${data['cost']:.6f}")

    print("\n✅ All clauses processed (async).")
    print(f"🔢 Total tokens used: {total_tokens}")
    print(f"💰 Total cost: ${total_cost:.6f}")

    return results

if __name__ == "__main__":
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    final_results = await run_all_clauses() if 'google.colab' in sys.modules or 'IPython' in sys.modules else asyncio.run(run_all_clauses())
    


/tmp/ipykernel_11059/2935787740.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))
/tmp/ipykernel_11059/2935787740.py:18: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = audit_chain.run(
/home/sherry/DocAnalyzer/Document_Analyzer_Streamlit_App/chains/audit_chain.py:51: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm, prompt=promp

🔎 Clause Processed: 19 controls
🧠 Tokens used: 32077
💵 Cost: $0.016247
🔎 Clause Processed: 19 controls
🧠 Tokens used: 31927
💵 Cost: $0.015960
🔎 Clause Processed: 19 controls
🧠 Tokens used: 31990
💵 Cost: $0.016154
🔎 Clause Processed: 18 controls
🧠 Tokens used: 32226
💵 Cost: $0.016589
🔎 Clause Processed: 18 controls
🧠 Tokens used: 31756
💵 Cost: $0.015816

✅ All clauses processed (async).
🔢 Total tokens used: 159976
💰 Total cost: $0.080766


In [11]:
final_results[0]

'[\n  {\n    "Clause": "4,5",\n    "Section": "A.5 (Organisational Controls)",\n    "Control Id": "A.5.1",\n    "Control Title": "Policies for information security",\n    "Compliance": "✅",\n    "Policy": "The Information Security Policy document defines baseline control measures, scope, enforcement, and roles. It establishes policies for confidentiality, asset management, access control, incident management, and compliance aligned with ISO 27001 standards.",\n    "Reference": "Information Security Policy [Internal].docx - Sections: Confidentiality Statement, Chapter 1 General, Section 1.1 Scope of the policy, Section 1.4 Enforcements, Chapter 10 Evaluation and Improvement (Section 10.1 Review of information security).",\n    "Gaps Identified": "",\n    "Recommended Action": ""\n  },\n  {\n    "Clause": "4,5",\n    "Section": "A.5 (Organisational Controls)",\n    "Control Id": "A.5.2",\n    "Control Title": "Information security roles and responsibilities",\n    "Compliance": "✅",\n   

In [12]:
evidence_chain = analyze_evidence(llm)
result = evidence_chain.run(
    text=evidence_text,
    control_json=final_results
)

In [13]:
result

'[\n  {\n    "Control Id": "A.5.25",\n    "Policy Implementation Evidence": "Evidence controls are applied/A.5.25/#18- CAPA-form [Confidential] - possible confidential sales data leaked during offboarding of employee .docx.pdf",\n    "Exact Evidence Extract": "An employee that was leaving involuntarily sent some confidential data to an external mail address. Investigation by system administrator confirmed 29 emails with customer names and business email addresses sent externally. Corrective actions included gathering forensic evidence, legal action, and updating the Information Security Policy to clarify rules on personal devices.",\n    "Evidence Recommendation": "None",\n    "Evidence": "✅"\n  },\n  {\n    "Control Id": "A.5.8",\n    "Policy Implementation Evidence": "Evidence controls are applied/A.5.8/4.4.1.6.1 System development life cycle [Confidential].pptx.pdf",\n    "Exact Evidence Extract": "Documented SDLC includes testing in CI/CD pipeline with unit, integration, vulnerabil

In [14]:
import json
results=result
with open("evidence_report.json", "w") as f:
    f.write(json.dumps(results, indent=2, ensure_ascii=False))

In [15]:
results

'[\n  {\n    "Control Id": "A.5.25",\n    "Policy Implementation Evidence": "Evidence controls are applied/A.5.25/#18- CAPA-form [Confidential] - possible confidential sales data leaked during offboarding of employee .docx.pdf",\n    "Exact Evidence Extract": "An employee that was leaving involuntarily sent some confidential data to an external mail address. Investigation by system administrator confirmed 29 emails with customer names and business email addresses sent externally. Corrective actions included gathering forensic evidence, legal action, and updating the Information Security Policy to clarify rules on personal devices.",\n    "Evidence Recommendation": "None",\n    "Evidence": "✅"\n  },\n  {\n    "Control Id": "A.5.8",\n    "Policy Implementation Evidence": "Evidence controls are applied/A.5.8/4.4.1.6.1 System development life cycle [Confidential].pptx.pdf",\n    "Exact Evidence Extract": "Documented SDLC includes testing in CI/CD pipeline with unit, integration, vulnerabil

In [16]:
final_results

['[\n  {\n    "Clause": "4,5",\n    "Section": "A.5 (Organisational Controls)",\n    "Control Id": "A.5.1",\n    "Control Title": "Policies for information security",\n    "Compliance": "✅",\n    "Policy": "The Information Security Policy document defines baseline control measures, scope, enforcement, and roles. It establishes policies for confidentiality, asset management, access control, incident management, and compliance aligned with ISO 27001 standards.",\n    "Reference": "Information Security Policy [Internal].docx - Sections: Confidentiality Statement, Chapter 1 General, Section 1.1 Scope of the policy, Section 1.4 Enforcements, Chapter 10 Evaluation and Improvement (Section 10.1 Review of information security).",\n    "Gaps Identified": "",\n    "Recommended Action": ""\n  },\n  {\n    "Clause": "4,5",\n    "Section": "A.5 (Organisational Controls)",\n    "Control Id": "A.5.2",\n    "Control Title": "Information security roles and responsibilities",\n    "Compliance": "✅",\n  

In [19]:
def load_json_report(results):
    # Step 1: Extract content from wrapper
    if isinstance(results, list):
        if len(results) == 1 and isinstance(results[0], str):
            raw_text = results[0]
        elif all(isinstance(item, dict) for item in results):
            return results
        else:
            raw_text = ''.join(results)
    elif isinstance(results, str):
        raw_text = results
    else:
        raw_text = str(results)
 
    # Step 2: Remove markdown fences
    raw_text = raw_text.strip()
    if raw_text.startswith("```json"):
        raw_text = raw_text.removeprefix("```json").strip()
    if raw_text.endswith("```"):
        raw_text = raw_text.removesuffix("```").strip()
 
    # Step 3: Use regex to extract all JSON arrays
    try:
        # Extract all top-level lists using regex
        arrays = re.findall(r'\[\s*{.*?}\s*\]', raw_text, re.DOTALL)
        combined = []
        for arr in arrays:
            data = json.loads(arr)
            if isinstance(data, list):
                combined.extend(data)
            else:
                combined.append(data)
        return combined
 
    except json.JSONDecodeError as e:
        print("❌ JSON parsing failed:", e)
        print("--- JSON Preview ---")
        print(raw_text[:500])
        raise

In [20]:
# Step 3: Convert to DataFrame
parsed_results = load_json_report(results)
parsed_final_results = load_json_report(final_results)
resutls_df = pd.DataFrame(parsed_results)
final_result_df= pd.DataFrame(parsed_final_results)

In [21]:
resutls_df

,Control Id,Policy Implementation Evidence,Exact Evidence Extract,Evidence Recommendation,Evidence
0,A.5.25,Evidence controls are applied/A.5.25/#18- CAPA...,An employee that was leaving involuntarily sen...,None,✅
1,A.5.8,Evidence controls are applied/A.5.8/4.4.1.6.1 ...,Documented SDLC includes testing in CI/CD pipe...,None,✅
2,A.5.22,Evidence controls are applied/A.5.22/Vendor ri...,Vendor risk assessment for Jumpcloud includes ...,None,✅


In [22]:
final_result_df

,Clause,Section,Control Id,Control Title,Compliance,Policy,Reference,Gaps Identified,Recommended Action
0,"4,5",A.5 (Organisational Controls),A.5.1,Policies for information security,✅,The Information Security Policy document defin...,Information Security Policy [Internal].docx - ...,,
1,"4,5",A.5 (Organisational Controls),A.5.2,Information security roles and responsibilities,✅,Roles and responsibilities are clearly defined...,Information Security Policy [Internal].docx - ...,,
2,5,A.5 (Organisational Controls),A.5.3,Segregation of duties,✅,Segregation of duties is mandated to reduce ri...,Information Security Policy [Internal].docx - ...,,
3,"5,6,9,10",A.5 (Organisational Controls),A.5.4,Management responsibilities,✅,Management is responsible for setting security...,Information Security Policy [Internal].docx - ...,,
4,,A.5 (Organisational Controls),A.5.5,Contact with authorities,✅,Procedures exist for contacting local authorit...,Business Continuity Plan E-WISE [Restricted].p...,,
...,...,...,...,...,...,...,...,...,...
87,,A.8 (Technological Controls),A.8.30,Outsourced development,✅,Outsourced development requires review of SDLC...,Section 8.1.1-8.1.4 and 3.1 Business Partnersh...,,
88,6,A.8 (Technological Controls),A.8.31,"Separation of development, test and production...",✅,"Development, test, and production environments...",Section 8.1.14-8.1.15 and 8.1.21 - Separation ...,,
89,8,A.8 (Technological Controls),A.8.32,Change management,✅,Change management is documented with strict re...,Section 6.2 Operational change control and 8.1...,,
90,,A.8 (Technological Controls),A.8.33,Test information,✅,"Test environments use limited, anonymised, or ...",Section 8.1.21 - Test environments only have l...,,


In [23]:
parsed_final_results

[{'Clause': '4,5',
  'Section': 'A.5 (Organisational Controls)',
  'Control Id': 'A.5.1',
  'Control Title': 'Policies for information security',
  'Compliance': '✅',
  'Policy': 'The Information Security Policy document defines baseline control measures, scope, enforcement, and roles. It establishes policies for confidentiality, asset management, access control, incident management, and compliance aligned with ISO 27001 standards.',
  'Reference': 'Information Security Policy [Internal].docx - Sections: Confidentiality Statement, Chapter 1 General, Section 1.1 Scope of the policy, Section 1.4 Enforcements, Chapter 10 Evaluation and Improvement (Section 10.1 Review of information security).',
  'Gaps Identified': '',
  'Recommended Action': ''},
 {'Clause': '4,5',
  'Section': 'A.5 (Organisational Controls)',
  'Control Id': 'A.5.2',
  'Control Title': 'Information security roles and responsibilities',
  'Compliance': '✅',
  'Policy': 'Roles and responsibilities are clearly defined for

In [24]:
merged_df = pd.merge(final_result_df, resutls_df, on="Control Id", how="outer")


In [25]:
merged_df['Evidence'] = merged_df['Evidence'].replace('', pd.NA).fillna('❌')

In [26]:
merged_df

,Clause,Section,Control Id,Control Title,Compliance,Policy,Reference,Gaps Identified,Recommended Action,Policy Implementation Evidence,Exact Evidence Extract,Evidence Recommendation,Evidence
0,"4,5",A.5 (Organisational Controls),A.5.1,Policies for information security,✅,The Information Security Policy document defin...,Information Security Policy [Internal].docx - ...,,,NaN,NaN,NaN,❌
1,6,A.5 (Organisational Controls),A.5.10,Acceptable use of information and other associ...,✅,Acceptable use policies are defined covering I...,Information Security Policy [Internal].docx - ...,,,NaN,NaN,NaN,❌
2,,A.5 (Organisational Controls),A.5.11,Return of assets,✅,Documented procedures require return of compan...,Information Security Policy [Internal].docx - ...,,,NaN,NaN,NaN,❌
3,6,A.5 (Organisational Controls),A.5.12,Classification of information,✅,Information classification schemes are establi...,Information Security Policy [Internal].docx - ...,,,NaN,NaN,NaN,❌
4,,A.5 (Organisational Controls),A.5.13,Labelling of information,✅,Information must be labelled and handled per c...,Information Security Policy [Internal].docx - ...,,,NaN,NaN,NaN,❌
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,,A.8 (Technological Controls),A.8.5,Secure authentication,✅,Section 4.1 Use of secret authentication infor...,Information Security Policy [Internal].docx - ...,,,NaN,NaN,NaN,❌
88,8,A.8 (Technological Controls),A.8.6,Capacity management,✅,Section 3.3.2 Contract Management includes mon...,Information Security Policy [Internal].docx - ...,,,NaN,NaN,NaN,❌
89,8,A.8 (Technological Controls),A.8.7,Protection against malware,✅,Section 6.6 Protection against malicious softw...,Information Security Policy [Internal].docx - ...,,,NaN,NaN,NaN,❌
90,8,A.8 (Technological Controls),A.8.8,Management of technical vulnerabilities,✅,Section 6.2 Management of technical vulnerabil...,Information Security Policy [Internal].docx - ...,,,NaN,NaN,NaN,❌


In [ ]:
merged_df.to_excel("result.xlsx")

In [ ]:
with open("output.md", "w", encoding="utf-8") as f:
    f.write(text)

print("✅ PDF content extracted and saved as 'output.md'")

✅ PDF content extracted and saved as 'output.md'
